# Importing and Setting up the API

In [1]:
# Note: this was necessary even though I installed spotipy via my cmd because the shell and this kernel use different versions
# of python. So in order to install spotipy here, we need to install it using sys.executable and not regular python3.
import sys
!{sys.executable} -m pip install spotipy


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: C:\Users\neela\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import cred

For our web app, we're going to use the **Client Credentials** flow. This project is relatively small in scale, so this flow allows us to access a user's data without the need to implement refresh tokens.

In [3]:
# Authenticating (without user credentials)
cid = cred.client_ID
secret = cred.client_SECRET

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager, requests_timeout=10)

In [4]:
# Random Test
weeknduri = 'spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ' # you don't need the 'spotify:artist:'
results = sp.artist_top_tracks(weeknduri) # dictionary
for i, track in enumerate(results['tracks'][:10]):
    print(f'{i+1}. ' + track['name'])

1. Die For You (with Ariana Grande) - Remix
2. Creepin' (with The Weeknd & 21 Savage)
3. Die For You
4. Starboy
5. Blinding Lights
6. Save Your Tears
7. I Was Never There
8. Reminder
9. Call Out My Name
10. Save Your Tears (Remix) (with Ariana Grande) - Bonus Track


In [5]:
playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF?si=1333723a6eff4b7f"
playlist_URI = playlist_link.split('/')[-1].split('?')[0] # extracts the URI from the link
playlist_URI

track_uris = []
for track in sp.playlist_tracks(playlist_URI)['items']:
    track_uris.append(track['track']['uri'])

# track_uris

In [6]:
sp.audio_features(track_uris[0]) # this is method we can use to get features of songs

[{'danceability': 0.707,
  'energy': 0.681,
  'key': 0,
  'loudness': -4.325,
  'mode': 1,
  'speechiness': 0.0668,
  'acousticness': 0.0632,
  'instrumentalness': 5.15e-06,
  'liveness': 0.0322,
  'valence': 0.646,
  'tempo': 117.999,
  'type': 'audio_features',
  'id': '4DHcnVTT87F0zZhRPYmZ3B',
  'uri': 'spotify:track:4DHcnVTT87F0zZhRPYmZ3B',
  'track_href': 'https://api.spotify.com/v1/tracks/4DHcnVTT87F0zZhRPYmZ3B',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4DHcnVTT87F0zZhRPYmZ3B',
  'duration_ms': 200455,
  'time_signature': 4}]

# Getting the data

In [7]:
import pandas as pd
import json

In [8]:
data_path = 'mpd.slice.0-999.json'
json_dict = json.loads(open(data_path).read()) # json.loads() deserializes a string representing json object and returns a python dictionary
# note: json.load(open(data_path)) would've done the same thing, loads works on strings and load works on file objects

playlists = json_dict['playlists'] # list of 
df = pd.json_normalize(playlists, record_path='tracks', meta='name') # record_path looks at 'tracks' in every playlist, meta grabs the playlist name

# df.to_csv('raw_data.csv')
df

# pd.read_json('mpd.slice.0-999.json').head()

,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,name
0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,Throwbacks
1,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone,Throwbacks
2,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit),Throwbacks
3,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified,Throwbacks
4,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot,Throwbacks
...,...,...,...,...,...,...,...,...,...
67498,39,James Arthur,spotify:track:5uCax9HTNlzGybIStD3vDh,spotify:artist:4IWBUUAFIplrNtaOHcJPRM,Say You Won't Let Go,spotify:album:7oiJYvEJHsmYtrgviAVIBD,211466,Back from the Edge,thinking of you
67499,40,Big Words,spotify:track:0P1oO2gREMYUCoOkzYAyFu,spotify:artist:0sHN89qak07mnug3LVVjzP,The Answer,spotify:album:5jrsRHRAmetu5e7RRBoxj7,263679,"Hollywood, a Beautiful Coincidence",thinking of you
67500,41,Allan Rayman,spotify:track:2oM4BuruDnEvk59IvIXCwn,spotify:artist:6Yv6OBXD6ZQakEljaGaDAk,25.22,spotify:album:3CbNgBzI7r9o0F6VjH9sTY,189213,Roadhouse 01,thinking of you
67501,42,Jon Jason,spotify:track:4Ri5TTUgjM96tbQZd5Ua7V,spotify:artist:77bNdkKYBBmc30CisCA6tE,Good Feeling,spotify:album:2dZ7oVNQBeLlpoUYfbEsJP,194720,Good Feeling,thinking of you


In [9]:
quick = pd.DataFrame(columns=["lengths"])
for i in range(len(playlists)):
    quick.loc[len(quick.index)] = len(playlists[i]['tracks'])

quick.iloc[:,0].mean(axis=0)

67.503

In [10]:
# Test
fd = pd.json_normalize(playlists, record_path='tracks')
fd.head()

,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook
1,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone
2,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit)
3,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified
4,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot


# Extracting the features from the data

In [11]:
data_path = 'raw_data.csv'
df2 = pd.read_csv(data_path)
# df2['track_uri'] = df2['track_uri'].apply(lambda x: re.findall(r'\w+$', x)[0]) Not necessary lmao
# df2 = df2[:100]
df2
# for numrow in range(len(df)):
#     df['track_uri'][i]
# print(df2.shape)

,Unnamed: 0,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,name
0,0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,Throwbacks
1,1,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone,Throwbacks
2,2,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit),Throwbacks
3,3,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified,Throwbacks
4,4,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot,Throwbacks
...,...,...,...,...,...,...,...,...,...,...
67498,67498,39,James Arthur,spotify:track:5uCax9HTNlzGybIStD3vDh,spotify:artist:4IWBUUAFIplrNtaOHcJPRM,Say You Won't Let Go,spotify:album:7oiJYvEJHsmYtrgviAVIBD,211466,Back from the Edge,thinking of you
67499,67499,40,Big Words,spotify:track:0P1oO2gREMYUCoOkzYAyFu,spotify:artist:0sHN89qak07mnug3LVVjzP,The Answer,spotify:album:5jrsRHRAmetu5e7RRBoxj7,263679,"Hollywood, a Beautiful Coincidence",thinking of you
67500,67500,41,Allan Rayman,spotify:track:2oM4BuruDnEvk59IvIXCwn,spotify:artist:6Yv6OBXD6ZQakEljaGaDAk,25.22,spotify:album:3CbNgBzI7r9o0F6VjH9sTY,189213,Roadhouse 01,thinking of you
67501,67501,42,Jon Jason,spotify:track:4Ri5TTUgjM96tbQZd5Ua7V,spotify:artist:77bNdkKYBBmc30CisCA6tE,Good Feeling,spotify:album:2dZ7oVNQBeLlpoUYfbEsJP,194720,Good Feeling,thinking of you


We'll add in the audio features as columns, as well as the artist's popularity, the genres of the song, and the track's popularity.

In [12]:
import re
def features(trackuri):
    featuredict = sp.audio_features(trackuri)[0]
    artistid = sp.track(trackuri)['artists'][0]['id']
    artistpop = sp.artist(artistid)['popularity'] # add in artist popularity
    featuredict['artistpop'] = artistpop
    trackpop = sp.track(trackuri)['popularity'] # add in track popularity
    featuredict['trackpop'] = trackpop
    genres = sp.artist(artistid)['genres']
#     genres_und = []
#     for genre in genres:
#         if ' ' in genre:
#             genres_und.append(genre.replace(' ', '_'))
#         else: genres_und.append(genre)
    if genres:
        featuredict['genres'] = ' '.join(re.sub(' ','_',genre) for genre in genres) # short version
    else: 
        featuredict['genres'] = 'unknown'

    return featuredict

# print(type(sp.artist('2wIVse2owClT7go1WT98tk')['popularity']))
# print(sp.artist('2wIVse2owClT7go1WT98tk')['genres'])
# print(sp.track('0UaMYEvWZi0ZqiDOoHU3YI')['artists'][0]['id'])
print(features('spotify:track:0UaMYEvWZi0ZqiDOoHU3YI'))

{'danceability': 0.904, 'energy': 0.813, 'key': 4, 'loudness': -7.105, 'mode': 0, 'speechiness': 0.121, 'acousticness': 0.0311, 'instrumentalness': 0.00697, 'liveness': 0.0471, 'valence': 0.81, 'tempo': 125.461, 'type': 'audio_features', 'id': '0UaMYEvWZi0ZqiDOoHU3YI', 'uri': 'spotify:track:0UaMYEvWZi0ZqiDOoHU3YI', 'track_href': 'https://api.spotify.com/v1/tracks/0UaMYEvWZi0ZqiDOoHU3YI', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0UaMYEvWZi0ZqiDOoHU3YI', 'duration_ms': 226864, 'time_signature': 4, 'artistpop': 74, 'trackpop': 68, 'genres': 'dance_pop hip_hop hip_pop pop_rap r&b rap urban_contemporary virginia_hip_hop'}


In [13]:
featurelist = [] # list of dictionaries

for i, track in enumerate(df2[:100]['track_uri']):
    if i%5==0: print(i, end=' ')
    try: featurelist.append(features(track))
    except: continue
featuredf = pd.DataFrame(featurelist)
featuredf.rename({"uri":"track_uri"}, axis='columns', inplace=True)
featuredf

0 5 10 15 20 25 30 35 40 45 50 55 60 65 70 75 80 85 90 95 

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,type,id,track_uri,track_href,analysis_url,duration_ms,time_signature,artistpop,trackpop,genres
0,0.904,0.813,4,-7.105,0,0.1210,0.03110,0.006970,0.0471,0.810,...,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864,4,74,68,dance_pop hip_hop hip_pop pop_rap r&b rap urba...
1,0.774,0.838,5,-3.914,0,0.1140,0.02490,0.025000,0.2420,0.924,...,audio_features,6I9VzXrHxO9rA9A5euc8Ak,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,https://api.spotify.com/v1/audio-analysis/6I9V...,198800,4,81,82,dance_pop pop
2,0.664,0.758,2,-6.583,0,0.2100,0.00238,0.000000,0.0598,0.701,...,audio_features,0WqIKmW4BTrj3eJFmnCKMv,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,https://api.spotify.com/v1/audio-analysis/0WqI...,235933,4,88,20,dance_pop pop r&b
3,0.892,0.714,4,-6.055,0,0.1410,0.20100,0.000234,0.0521,0.817,...,audio_features,1AWQoqb9bSvzTjaLralEkT,spotify:track:1AWQoqb9bSvzTjaLralEkT,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,https://api.spotify.com/v1/audio-analysis/1AWQ...,267267,4,81,77,dance_pop pop
4,0.853,0.606,0,-4.596,1,0.0713,0.05610,0.000000,0.3130,0.654,...,audio_features,1lzr43nnXAijIGYnCT8M8H,spotify:track:1lzr43nnXAijIGYnCT8M8H,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,https://api.spotify.com/v1/audio-analysis/1lzr...,227600,4,74,0,dance_pop pop_rap reggae_fusion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.539,0.846,8,-3.192,1,0.0354,0.00630,0.000002,0.1300,0.460,...,audio_features,0WNGsQ1oAuHzNTk8jivBKW,spotify:track:0WNGsQ1oAuHzNTk8jivBKW,https://api.spotify.com/v1/tracks/0WNGsQ1oAuHz...,https://api.spotify.com/v1/audio-analysis/0WNG...,274097,4,92,1,k-pop k-pop_boy_group
96,0.729,0.898,5,-1.029,1,0.0391,0.31500,0.000000,0.1070,0.517,...,audio_features,24psBRmEw3kHjBGZfl1dmb,spotify:track:24psBRmEw3kHjBGZfl1dmb,https://api.spotify.com/v1/tracks/24psBRmEw3kH...,https://api.spotify.com/v1/audio-analysis/24ps...,218475,4,43,0,k-pop k-pop_girl_group
97,0.609,0.246,8,-8.449,1,0.0376,0.78400,0.000000,0.0825,0.371,...,audio_features,06L1apH8kLF47dbhZ4Zg9A,spotify:track:06L1apH8kLF47dbhZ4Zg9A,https://api.spotify.com/v1/tracks/06L1apH8kLF4...,https://api.spotify.com/v1/audio-analysis/06L1...,288993,4,68,0,k-pop pop
98,0.763,0.658,5,-5.643,0,0.1470,0.02580,0.000000,0.0915,0.389,...,audio_features,2qWgqPdW1OiAP8KSBH1b93,spotify:track:2qWgqPdW1OiAP8KSBH1b93,https://api.spotify.com/v1/tracks/2qWgqPdW1OiA...,https://api.spotify.com/v1/audio-analysis/2qWg...,217861,4,68,0,k-pop pop


In [14]:
# df2small = df2.head(100)
# final_100 = pd.merge(df2small, featuredf, how='outer')
# final_100
featuredf.head(2)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,type,id,track_uri,track_href,analysis_url,duration_ms,time_signature,artistpop,trackpop,genres
0,0.904,0.813,4,-7.105,0,0.121,0.0311,0.00697,0.0471,0.810,...,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864,4,74,68,dance_pop hip_hop hip_pop pop_rap r&b rap urba...
1,0.774,0.838,5,-3.914,0,0.114,0.0249,0.02500,0.2420,0.924,...,audio_features,6I9VzXrHxO9rA9A5euc8Ak,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,https://api.spotify.com/v1/audio-analysis/6I9V...,198800,4,81,82,dance_pop pop


In [15]:
df2.head(2)

,Unnamed: 0,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,name
0,0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,Throwbacks
1,1,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone,Throwbacks


In [18]:
master = pd.merge(df2[:100], featuredf, on='track_uri')
master.drop(columns=['Unnamed: 0','pos'], inplace=True)
# cols = master.columns.tolist()
# cols = [cols[7]] + cols[0:7] + cols[8:]
# master = master[cols]

In [19]:
# master.to_csv('out2.csv')
print(master.shape)
print(df2.shape)
print(featuredf.shape)

(102, 28)
(67503, 10)
(100, 21)


In [20]:
master

,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms_x,album_name,name,danceability,energy,...,tempo,type,id,track_href,analysis_url,duration_ms_y,time_signature,artistpop,trackpop,genres
0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,Throwbacks,0.904,0.813,...,125.461,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864,4,74,68,dance_pop hip_hop hip_pop pop_rap r&b rap urba...
1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone,Throwbacks,0.774,0.838,...,143.040,audio_features,6I9VzXrHxO9rA9A5euc8Ak,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,https://api.spotify.com/v1/audio-analysis/6I9V...,198800,4,81,82,dance_pop pop
2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit),Throwbacks,0.664,0.758,...,99.259,audio_features,0WqIKmW4BTrj3eJFmnCKMv,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,https://api.spotify.com/v1/audio-analysis/0WqI...,235933,4,88,20,dance_pop pop r&b
3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified,Throwbacks,0.892,0.714,...,100.972,audio_features,1AWQoqb9bSvzTjaLralEkT,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,https://api.spotify.com/v1/audio-analysis/1AWQ...,267267,4,81,77,dance_pop pop
4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot,Throwbacks,0.853,0.606,...,94.759,audio_features,1lzr43nnXAijIGYnCT8M8H,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,https://api.spotify.com/v1/audio-analysis/1lzr...,227600,4,74,0,dance_pop pop_rap reggae_fusion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,BTS,spotify:track:0WNGsQ1oAuHzNTk8jivBKW,spotify:artist:3Nrfpe0tUJi4K4DXYWgMUX,Spring Day,spotify:album:7LF4N7lvyDhrPBuCJ1rplJ,274097,You Never Walk Alone,korean,0.539,0.846,...,106.992,audio_features,0WNGsQ1oAuHzNTk8jivBKW,https://api.spotify.com/v1/tracks/0WNGsQ1oAuHz...,https://api.spotify.com/v1/audio-analysis/0WNG...,274097,4,92,1,k-pop k-pop_boy_group
98,Lovelyz,spotify:track:24psBRmEw3kHjBGZfl1dmb,spotify:artist:3g34PW5oNmDBxMVUTzx2XK,Ah-Choo,spotify:album:5ZJuawNI3RvxURIBtsDHs0,218474,Lovelyz8,korean,0.729,0.898,...,126.977,audio_features,24psBRmEw3kHjBGZfl1dmb,https://api.spotify.com/v1/tracks/24psBRmEw3kH...,https://api.spotify.com/v1/audio-analysis/24ps...,218475,4,43,0,k-pop k-pop_girl_group
99,LEE HI,spotify:track:06L1apH8kLF47dbhZ4Zg9A,spotify:artist:7cVZApDoQZpS447nHTsNqu,BREATHE,spotify:album:1xnXVzinhfO4I9CzTocPfh,288992,SEOULITE,korean,0.609,0.246,...,123.773,audio_features,06L1apH8kLF47dbhZ4Zg9A,https://api.spotify.com/v1/tracks/06L1apH8kLF4...,https://api.spotify.com/v1/audio-analysis/06L1...,288993,4,68,0,k-pop pop
100,LEE HI,spotify:track:2qWgqPdW1OiAP8KSBH1b93,spotify:artist:7cVZApDoQZpS447nHTsNqu,FXXK WIT US,spotify:album:1xnXVzinhfO4I9CzTocPfh,217861,SEOULITE,korean,0.763,0.658,...,80.038,audio_features,2qWgqPdW1OiAP8KSBH1b93,https://api.spotify.com/v1/tracks/2qWgqPdW1OiA...,https://api.spotify.com/v1/audio-analysis/2qWg...,217861,4,68,0,k-pop pop
